# EASY BLACKJACK PROJECT

We want to implement this little side project were we implement an agent playing blackjack against the house. 
First of all we implement an easy deck class to simulate the deck behaviour. It is really simplified but long story short we use a full deck everytime and when there are too less cards in the deck we use a new one.
We implemented in italian the name of the cards because they sound better.

In [18]:
import random
class Deck():
    def __init__(self):
        self.__seed = ['cuori', 'quadri', 'fiori', 'picche']
        self.__values = ['asso', 'due', 'tre', 'quattro', 'cinque', 'sei', 'sette', 'otto', 'nove', 'dieci', 'jack', 'regina', 're']
        self.__integer_values = {v : i + 1 for i, v in enumerate(self.__values)}
        self.__integer_values['jack'] = 10
        self.__integer_values['regina'] = 10
        self.__integer_values['re'] = 10
        self.__ordered_card = {c : i for i, c in enumerate([f"{v} di {s}" for s in self.__seed for v in self.__values])}
        self.__ordered_value = {i : c for i, c in enumerate([f"{v} di {s}" for s in self.__seed for v in self.__values])}
        self.__cards = [f"{v} di {s}" for s in self.__seed for v in self.__values ]

    def new_deck(self):
        self.__cards = [f"{v} di {s}" for v in self.__values for s in self.__seed]

    def shuffle(self):
        random.shuffle(self.__cards)

    def get_one(self):
        return self.__cards.pop()
    
    def get_some(self, k = 2):
        if k > 52:
            print("Maximum card requested 52")
            raise Exception
        if k > len(self.__cards):
            print("A new deck will be used") # just to remember
            self.new_deck()
        res = [self.get_one() for i in range (k)]
        return res
    
    def get_value_from_card(self, c):
        return self.__ordered_card[c]
    
    def get_card_from_value(self, v):
        return self.__ordered_value[v]

    def get_integer_value(self, c):
        v = c.split()[0]
        return self.__integer_values[v]
    
    def get_deck(self):
        return self.__cards

Now that the deck is working we build the house. The house is like a player but with a fixed policy (soft 17). The policy states the following : if hour hand value is less than 17, we ask for an other card. Otherwise we stay. It assign 1 the value 1 or 11 respectively based on what he can actually do (1 if 11 is too much, 11 otherwise).

In [194]:
class House:

    def __init__(self):
        self.hand = []
        self.hand_v = []
        self.value = 0

    def first(self, deck : Deck, verbose = True):
        self.hand = deck.get_some(2)
        self.hand_v = [deck.get_integer_value(c) for c in self.hand]
        if self.hand_v[0] == 1:
            self.hand_v[0] = 11
        if self.hand_v[1] == 1 and self.hand_v[0] != 11:
            self.hand_v[1] = 11
        if verbose:
            print(self.hand[0])
        return self.hand_v[0]

    def play(self, deck : Deck, verbose = True):
        if verbose:
            print(self.hand[1])
        # Soft 17 : under 17 hit, over it stare
        self.value = sum(self.hand_v)
        while self.value < 17:
            card = deck.get_one()
            if verbose:
                print(card)
            if deck.get_integer_value(card) == 1:
                if self.value + 11 <= 21:
                    self.hand.append(card)
                    self.hand_v.append(11)
                    self.value += 11
                else:
                    self.hand.append(card)
                    self.hand_v.append(deck.get_integer_value(card))
                    self.value = sum(self.hand_v)
            else:
                self.hand.append(card)
                self.hand_v.append(deck.get_integer_value(card))
                self.value = sum(self.hand_v)
        return self.value

In [337]:
d = Deck()
d.shuffle()
h = House()
h.first(d)
h.play(d)

sei di picche
jack di fiori
cinque di quadri


21

Now we need to implement the player. The player will be a simple agent where we will implement some value iteration (for now).